Here, I need to add rows for all non-orders I.e. if an item was ordered in order 1 and not order 2, there should be a new row with all the same order details (day, time, etc.) but the product that was previously ordered and a 0 in add_to_cart_sequence (& 'reordered') to indicate a non-order. This needs to be done for every subsequent order, so that by a user's final order, products included are everything they bought on that final order and rows indicating non-orders for all products they ever ordered. 

In [3]:
import pandas as pd
import numpy as np
import os
import random

# from library.sb_utils import save_file

In [4]:
df = pd.read_csv('first_subset.csv')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1907 entries, 0 to 1906
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   order_id                1907 non-null   int64  
 1   user_id                 1907 non-null   int64  
 2   order_by_user_sequence  1907 non-null   int64  
 3   order_dow               1907 non-null   int64  
 4   order_hour_of_day       1907 non-null   int64  
 5   days_since_prior_order  1907 non-null   float64
 6   add_to_cart_sequence    1907 non-null   int64  
 7   reordered               1907 non-null   int64  
 8   product_name            1907 non-null   object 
 9   aisle_name              1907 non-null   object 
 10  dept_name               1907 non-null   object 
dtypes: float64(1), int64(7), object(3)
memory usage: 164.0+ KB


In [6]:
df.head()

,order_id,user_id,order_by_user_sequence,order_dow,order_hour_of_day,days_since_prior_order,add_to_cart_sequence,reordered,product_name,aisle_name,dept_name
0,1960542,604,1,1,8,-1.0,2,0,Bag of Organic Bananas,fresh fruits,produce
1,1960542,604,1,1,8,-1.0,8,0,Organic Raspberries,packaged vegetables fruits,produce
2,1960542,604,1,1,8,-1.0,5,0,Organic Bosc Pear,fresh fruits,produce
3,1960542,604,1,1,8,-1.0,9,0,Globe Eggplant,fresh vegetables,produce
4,1960542,604,1,1,8,-1.0,1,0,Organic Reduced Fat Milk,milk,dairy eggs


In [7]:
df['user_id'].value_counts()

604       420
76977     317
30858     238
199739    134
205651    130
1856      105
150397     94
37419      78
146419     64
9862       62
194888     61
148025     44
142628     33
41207      31
89907      29
41954      26
49134      15
67355      13
54129       9
143034      4
Name: user_id, dtype: int64

In [8]:
ini = df[df['order_by_user_sequence'] == 1]
sec = df[df['order_by_user_sequence'] != 1]

print(ini.shape)
print(sec.shape)

(212, 11)
(1695, 11)


In [9]:
ini_gb = ini.groupby(['user_id', 'product_name'])['order_id'].count()
ini_gb = ini_gb.reset_index()
ini_gb = ini_gb.rename({'order_id': 'count'}, axis=1) 
ini_gb

,user_id,product_name,count
0,604,Apple Honeycrisp Organic,1
1,604,Bag of Organic Bananas,1
2,604,Globe Eggplant,1
3,604,"Granola Bars, Crunchy, Macaroon Crunch",1
4,604,Minced Ginger,1
...,...,...,...
207,205651,Smooth Ranch Dip,1
208,205651,Strawberries,1
209,205651,Strawberry Preserves,1
210,205651,Watermelon/Mixed Berry Variety Pack Low-Fat Gr...,1


In [10]:
sec_gb = sec.groupby(['user_id', 'order_by_user_sequence', 'product_name'])['order_id'].count()
sec_gb = sec_gb.reset_index()
sec_gb = sec_gb.rename({'order_id': 'count'}, axis=1) 
sec_gb

,user_id,order_by_user_sequence,product_name,count
0,604,2,100% Pure Avocado Oil,1
1,604,2,Apple Honeycrisp Organic,1
2,604,2,Cane Sugar,1
3,604,2,Natural Artesian Water,1
4,604,2,Organic All Vegetable Shortening,1
...,...,...,...,...
1690,205651,14,Grapefruit Sparkling Water,1
1691,205651,14,Jet Puffed Miniature Marshmallows,1
1692,205651,14,Lemonade,1
1693,205651,14,No Pulp Calcium & Vitamin D Pure Orange Juice,1


In [11]:
already_ordered1 = pd.merge(ini_gb, sec_gb, how='inner', on=['user_id', 'product_name'])
already_ordered1.head(10)

,user_id,product_name,count_x,order_by_user_sequence,count_y
0,604,Apple Honeycrisp Organic,1,2,1
1,604,Apple Honeycrisp Organic,1,3,1
2,604,Apple Honeycrisp Organic,1,16,1
3,604,Apple Honeycrisp Organic,1,17,1
4,604,Apple Honeycrisp Organic,1,19,1
5,604,Bag of Organic Bananas,1,3,1
6,604,Bag of Organic Bananas,1,4,1
7,604,Bag of Organic Bananas,1,6,1
8,604,Bag of Organic Bananas,1,7,1
9,604,Bag of Organic Bananas,1,13,1


In [12]:
df[df['product_name'].isin(['Apple Honeycrisp Organic', 'Bag of Organic Bananas'])].head(7)

,order_id,user_id,order_by_user_sequence,order_dow,order_hour_of_day,days_since_prior_order,add_to_cart_sequence,reordered,product_name,aisle_name,dept_name
0,1960542,604,1,1,8,-1.0,2,0,Bag of Organic Bananas,fresh fruits,produce
5,1960542,604,1,1,8,-1.0,4,0,Apple Honeycrisp Organic,fresh fruits,produce
14,2288851,604,2,4,11,10.0,5,1,Apple Honeycrisp Organic,fresh fruits,produce
25,2715640,604,3,2,8,12.0,5,1,Bag of Organic Bananas,fresh fruits,produce
29,2715640,604,3,2,8,12.0,2,1,Apple Honeycrisp Organic,fresh fruits,produce
51,48663,604,4,6,9,4.0,5,1,Bag of Organic Bananas,fresh fruits,produce
92,699145,604,6,4,9,9.0,1,1,Bag of Organic Bananas,fresh fruits,produce


In [13]:
# This successfully recognizes all items *reordered* from order 1 in each subsequent order.
# Now use these dfs to *add* rows for *NON-orders*

prior_only = ini_gb.merge(sec_gb, on=['user_id', 'product_name'], how='left', indicator='True')
prior_only = prior_only.loc[prior_only['True']=='left_only',:]
prior_only

,user_id,product_name,count_x,order_by_user_sequence,count_y,True
21,604,"Granola Bars, Crunchy, Macaroon Crunch",1,NaN,NaN,left_only
33,604,Organic Chopped Garlic,1,NaN,NaN,left_only
35,604,Organic Red Bell Pepper,1,NaN,NaN,left_only
59,1856,Baby Food Pears Squash,1,NaN,NaN,left_only
65,1856,Classic Round Crackers,1,NaN,NaN,left_only
...,...,...,...,...,...,...
409,205651,Red Edition,1,NaN,NaN,left_only
410,205651,Smooth Ranch Dip,1,NaN,NaN,left_only
412,205651,Strawberry Preserves,1,NaN,NaN,left_only
413,205651,Watermelon/Mixed Berry Variety Pack Low-Fat Gr...,1,NaN,NaN,left_only


This is my understanding of a code for an anti-join. The only problem is that it returns items from order 1 that were reordered *never*, but there were items reordered in order 2 but not in order 3, vice-versa, etc. I need, for each order_by_user_sequence, to know which items from order 1 were *not* ordered at all *in that specific order*

In [ ]:
# This was a method I tried previously. It worked fine for adding new rows to just a single 
# user, for all of their orders. But I haven't got it working yet when looping over users. 

# for user in users1:
    # Work with 1 user at a time
    rows_to_work_w = df1.loc[df1['user_id']==user,:].copy()
    for n in range(2,101):
        
        # Get items from prior order not yet in order n
        prior_order_items = set(rows_to_work_w[rows_to_work_w[
            'order_by_user_sequence']==(n-1)]['product_name'].
                                unique().tolist())
        order_n_items = set(df[df['order_by_user_sequence']==n][
            'product_name'].unique().tolist())
        not_yet_in_n = prior_order_items-order_n_items
        
        # Specify new rows as copies of rows from prior order...
        new_rows = rows_to_work_w.loc[rows_to_work_w[
            'order_by_user_sequence']==(n-1),:].copy()
        # ... where the product was not ordered in order n
        new_rows = new_rows.loc[new_rows['product_name'].isin(
            not_yet_in_n),:].copy()
        
        # Change value of add_to_cart_sequence and reordered to 0
        new_rows[['add_to_cart_sequence', 'reordered']] = 0
        # And change value of order_by_user_sequence from prior to n
        new_rows['order_by_user_sequence'] = n

        # Add these rows to user's rows to work with so they get 
        # included in next iteration of orders to be repeated 
        rows_to_work_w = pd.concat([rows_to_work_w, new_rows])
    
    # Once all this user's new rows are added, put just them in df1
    this_user_new_rows = rows_to_work_w.loc[rows_to_work_w[
        'add_to_cart_sequence']==0, :].copy()
    df1 = pd.concat([df1, this_user_new_rows])

Audio(sound_file, autoplay=True)

In [ ]:
df1.info()

In [ ]:
# Check to make sure that worked (each order>1 should have new rows
# with add_to_cart_sequence=0 for all items ever ordered previously)
df1.sample(20)

New rows take a long time to create. Immediately save as a file and work with that file in a new notebook so I have the ability to restart the kernel without having it take forever to run as I continue manipulation. Repeat with each df1, df2, df3, etc. and then concatenate them in the next notebook before moving on. 

In [ ]:
users_incorrect_deets = df1
datapath = '../data/processed'
save_file(users_incorrect_deets, 'users_incorrect_deets.csv', datapath)

Continue this in preprocessing1b: https://github.com/fractaldatalearning/Capstone2/blob/main/notebooks/preprocessing1b_get_usable_df.ipynb